In [ ]:
from glob import glob
import numpy as np
import math
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from astropy.io import fits
from scripts.visualization_helpers import *
import jwst
print(jwst.__version__)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib as mpl
%matplotlib inline

# figure sizes and resolutions
rcParams['figure.figsize'] = [10,10]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

cmap = plt.get_cmap('summer')
cmaplist = [cmap(i) for i in range(cmap.N)]

# create the new map
cmap = cmap.from_list('Custom cmap', cmaplist[0:], cmap.N)


In [ ]:
PROPOSAL_ID = '1386'
INSTRUMENT = 'NIRCAM'

In [ ]:
directory_1386_nircam = f'/data/scratch/bariskurtkaya/dataset/{INSTRUMENT}/{PROPOSAL_ID}/mastDownload/JWST/'

In [ ]:
fits_visualizer_psfstack = FITSVisualizer(directory=directory_1386_nircam, suffix='psfstack')
fits_visualizer_i2d = FITSVisualizer(directory=directory_1386_nircam, suffix='i2d')
fits_visualizer_psfsub =  FITSVisualizer(directory=directory_1386_nircam, suffix='psfsub')

In [ ]:
#fits_visualizer_i2d.plot()

In [ ]:
fits_visualizer_psfsub.plot()